<a href="https://colab.research.google.com/github/BhargavTelu/LSTM_shakespeare/blob/main/LSTM_NextWord.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [9]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
import nltk
nltk.download("gutenberg")
from nltk.corpus import gutenberg

[nltk_data] Downloading package gutenberg to /root/nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!


In [10]:
# load the data set
data = gutenberg.raw("shakespeare-hamlet.txt")

with open("hamlet.txt","w") as file:
  file.write(data)


### data preprocessing

In [11]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer

#load file
with open("hamlet.txt","r") as file:
  text= file.read().lower()

In [12]:
#tokenize the text
tokenizer = Tokenizer()
tokenizer.fit_on_texts([text]) #coverting the file into tokens
total_words = len(tokenizer.word_index)+1
total_words

4818

In [13]:
#create input sequences
input_sequence = []
for line in text.split("\n"):
  token_list = tokenizer.texts_to_sequences([line])[0]
  for i in range(1,len(token_list)):
    n_gram_sequence = token_list[:i+1]
    input_sequence.append(n_gram_sequence)

In [14]:
# maximum length of the text
max_input_sequence = max([len(x) for x in input_sequence])
max_input_sequence

14

In [15]:
from sys import maxsize
input_sequence = np.array(pad_sequences(input_sequence,maxlen= max_input_sequence, padding='pre'))
input_sequence

array([[   0,    0,    0, ...,    0,    1,  687],
       [   0,    0,    0, ...,    1,  687,    4],
       [   0,    0,    0, ...,  687,    4,   45],
       ...,
       [   0,    0,    0, ...,    4,   45, 1047],
       [   0,    0,    0, ...,   45, 1047,    4],
       [   0,    0,    0, ..., 1047,    4,  193]], dtype=int32)

In [16]:
x,y = input_sequence[:,:-1], input_sequence[:,-1]
y

array([ 687,    4,   45, ..., 1047,    4,  193], dtype=int32)

In [17]:
# coverting the y value to 0 or 1
y = tf.keras.utils.to_categorical(y,num_classes=total_words)
y

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [18]:
# splitting the data into train and test
x_train,x_test, y_train, y_test = train_test_split(x,y, random_state=42, test_size=0.2)

### train the model

In [19]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding,Dense, LSTM, Dropout

#define the model
model = Sequential()
model.add(Embedding(total_words, 100, input_length= max_input_sequence-1))
model.add(LSTM(150, return_sequences= True))
model.add(Dropout(0.2))
model.add(LSTM(150))
model.add(Dense(total_words, activation = "softmax"))

# compile the model
model.compile(optimizer = "adam", metrics = ["accuracy"], loss="categorical_crossentropy")
model.summary()


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_1 (LSTM)                        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [20]:
# train the model
history = model.fit(x_train,y_train,epochs=20, validation_data=(x_test,y_test), verbose = 1)

Epoch 1/20
644/644 ━━━━━━━━━━━━━━━━━━━━ 58s 76ms/step - accuracy: 0.0286 - loss: 7.1585 - val_accuracy: 0.0371 - val_loss: 6.6730
Epoch 2/20
644/644 ━━━━━━━━━━━━━━━━━━━━ 79s 73ms/step - accuracy: 0.0382 - loss: 6.4320 - val_accuracy: 0.0431 - val_loss: 6.7499
Epoch 3/20
644/644 ━━━━━━━━━━━━━━━━━━━━ 85s 77ms/step - accuracy: 0.0429 - loss: 6.2958 - val_accuracy: 0.0470 - val_loss: 6.7892
Epoch 4/20
644/644 ━━━━━━━━━━━━━━━━━━━━ 79s 73ms/step - accuracy: 0.0510 - loss: 6.1550 - val_accuracy: 0.0505 - val_loss: 6.8219
Epoch 5/20
644/644 ━━━━━━━━━━━━━━━━━━━━ 82s 72ms/step - accuracy: 0.0525 - loss: 6.0003 - val_accuracy: 0.0515 - val_loss: 6.8615
Epoch 6/20
644/644 ━━━━━━━━━━━━━━━━━━━━ 82s 73ms/step - accuracy: 0.0631 - loss: 5.8662 - val_accuracy: 0.0649 - val_loss: 6.9205
Epoch 7/20
644/644 ━━━━━━━━━━━━━━━━━━━━ 83s 74ms/step - accuracy: 0.0769 - loss: 5.6731 - val_accuracy: 0.0674 - val_loss: 6.9557
Epoch 8/20
644/644 ━━━━━━━━━━━━━━━━━━━━ 81s 73ms/step - accuracy: 0.0803 - loss: 5.5422 - 

## prediction

In [21]:
# function to predict the next word
def predict_next_word(model, tokenizer, text, max_input_sequence):
  token_list = tokenizer.texts_to_sequences([text])[0]
  if len(token_list) >= max_input_sequence:
    token_list = token_list[-(max_input_sequence-1):] # ensure the sequence length matches max_sequence
  token_list = pad_sequences([token_list], maxlen = max_input_sequence)
  predicted = model.predict(token_list, verbose = 0)
  predict_word_index = np.argmax(predicted, axis = 1)
  for word, index in tokenizer.word_index.items():
    if index == predict_word_index:
      return word
  return None

In [22]:
input_text = "To be or not to be"
print(f"Input text: {input_text}")
max_input_sequence = model.input_shape[1]+1
next_word = predict_next_word(model,tokenizer, input_text, max_input_sequence)
print(f"Next word prediction:{next_word}")

Input text: To be or not to be
Next word prediction:buried


In [23]:
# save the model
model.save("next_word_lstm.h5")
# save the tokenizer
import pickle
with open("tokenizer.pickle","wb") as handle:
  pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)
